In [98]:
import requests
from bs4 import BeautifulSoup
import os


url = 'http://www.cartoonmad.com/comic/5540.html'

header_s={
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    'Cookie': 'ASPSESSIONIDAQQSDBBQ=ILCIIGDCJJDGCNDPPJHNDCHI; __utma=42881207.196417283.1522300799.1522300799.1522300799.1; __utmc=42881207; __utmz=42881207.1522300799.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utmt=1; chname=%7C%A5j%A8%A3%A6P%BE%C7%A6%B3%A5%E6%ACy%BB%D9%C3%AA%AFg%7C%A4C%AD%EC%B8o%7C%AE%FC%B8%E9%A4%FD; chid=%7C05540%7C02504%7C01152; __utmb=42881207.6.10.1522300799',
    'Host': 'www.cartoonmad.com',
    'Pragma': 'no-cache',
    'Referer': 'http://www.cartoonmad.com/comic/554000972009001.html',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'
}

res = requests.get(url, headers=header_s)
res.encoding = 'big5'
res.text
soup = BeautifulSoup(res.text, "lxml")
selector_list = soup.select('#info table tr td a[href^="/comic/"]')
selector_list[0]['href']
#res.text


'/comic/554000012007001.html'

In [108]:
def get_imgs_links(url, header_s):
    imgs_links = []
    res = requests.get(url, headers=header_s)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, "lxml")
    selector_list = soup.select('select option[value]')
    imgs_num = len(selector_list)
    print('this monga total page number = %s'%str(imgs_num))
    url_template = soup.select('a img[src^=http://web.cartoonmad.com]')[0]['src']
    pat = re.findall('\d{3}.jpg', url_template)[0]
    url_template = url_template.replace(pat, '')
    print('url template : %s'%url_template)
    for num in range(1, imgs_num+1):
        num_str = '{:03}'.format(num)+'.jpg'
        link = url_template+num_str
        imgs_links.append(link)
        print(link)
    return imgs_links




In [109]:
def get_one_monga(url, header_s):
    imgs_links = get_imgs_links(url, header_s)
    res = requests.get(url, headers=header_s)
    res.encoding = 'big5'
    soup = BeautifulSoup(res.text, "lxml")
    title = soup.select('center li a')[0].text
    epi = soup.select('center li a')[1].text.replace(' 話', '')
    dir_name = title+epi
    print('this monga title is : '+dir_name)
    os.makedirs('crawl_mongas/'+dir_name, exist_ok=True)
    for num in range(len(imgs_links)):
        filename = 'crawl_mongas/'+dir_name+'/'+'p_'+str(num+1)+'.jpg'
        img_data = requests.get(imgs_links[num], headers=header_s).content
        with open(filename, 'wb') as f:
            f.write(img_data)
        print('[INFO]Done download page %s'%filename)
    print('[INFO]Done download monga %s'%dir_name)
            

In [110]:
def get_series_mongas(url, header_s):
    res = requests.get(url, headers=header_s)
    res.encoding = 'big5'
    soup = BeautifulSoup(res.text, "lxml")
    selector_list = soup.select('#info table tr td a[href^="/comic/"]')
    epi_num = len(selector_list)
    print('[START]this monga has %s epis'%str(epi_num))
    for num in range(epi_num):
        img_pg_link = 'http://www.cartoonmad.com'+selector_list[num]['href']
        print('image page link = %s'%img_pg_link)
        get_one_monga(img_pg_link, header_s)
    print('Done download all mongas!')


In [111]:
url = 'http://www.cartoonmad.com/comic/5540.html'
get_series_mongas(url, header_s)

[START]this monga has 97 epis
image page link = http://www.cartoonmad.com/comic/554000012007001.html
this monga total page number = 7
url template : http://web.cartoonmad.com/c8kcs73hv62/5540/001/
http://web.cartoonmad.com/c8kcs73hv62/5540/001/001.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/001/002.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/001/003.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/001/004.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/001/005.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/001/006.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/001/007.jpg
this monga title is : 古見同學有交流障礙症漫畫1
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫1/p_1.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫1/p_2.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫1/p_3.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫1/p_4.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫1/p_5.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫1/p_6.jpg
[INFO]Done download pa

[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫8/p_4.jpg
[INFO]Done download monga 古見同學有交流障礙症漫畫8
image page link = http://www.cartoonmad.com/comic/554000092012001.html
this monga total page number = 12
url template : http://web.cartoonmad.com/c8kcs73hv62/5540/009/
http://web.cartoonmad.com/c8kcs73hv62/5540/009/001.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/009/002.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/009/003.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/009/004.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/009/005.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/009/006.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/009/007.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/009/008.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/009/009.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/009/010.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/009/011.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/009/012.jpg
this monga title is : 古見同學有交流障礙症漫畫9
[INFO]Done download page crawl_monga

this monga title is : 古見同學有交流障礙症漫畫14
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫14/p_1.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫14/p_2.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫14/p_3.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫14/p_4.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫14/p_5.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫14/p_6.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫14/p_7.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫14/p_8.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫14/p_9.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫14/p_10.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫14/p_11.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫14/p_12.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫14/p_13.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫14/p_14.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫14/p_15.jpg
[INFO]Done download page crawl_mongas/古見同學

[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫19/p_5.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫19/p_6.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫19/p_7.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫19/p_8.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫19/p_9.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫19/p_10.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫19/p_11.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫19/p_12.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫19/p_13.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫19/p_14.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫19/p_15.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫19/p_16.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫19/p_17.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫19/p_18.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫19/p_19.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫19/p_20.jpg
[INFO]Done do

this monga title is : 古見同學有交流障礙症漫畫24
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫24/p_1.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫24/p_2.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫24/p_3.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫24/p_4.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫24/p_5.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫24/p_6.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫24/p_7.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫24/p_8.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫24/p_9.jpg
[INFO]Done download monga 古見同學有交流障礙症漫畫24
image page link = http://www.cartoonmad.com/comic/554000252020001.html
this monga total page number = 20
url template : http://web.cartoonmad.com/c8kcs73hv62/5540/025/
http://web.cartoonmad.com/c8kcs73hv62/5540/025/001.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/025/002.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/025/003.jpg
http://web.cartoonmad.com/c8kcs73hv62/55

...

this monga title is : 古見同學有交流障礙症漫畫77
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫77/p_1.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫77/p_2.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫77/p_3.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫77/p_4.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫77/p_5.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫77/p_6.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫77/p_7.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫77/p_8.jpg
[INFO]Done download monga 古見同學有交流障礙症漫畫77
image page link = http://www.cartoonmad.com/comic/554000782007001.html
this monga total page number = 7
url template : http://web.cartoonmad.com/c8kcs73hv62/5540/078/
http://web.cartoonmad.com/c8kcs73hv62/5540/078/001.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/078/002.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/078/003.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/078/004.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/078/

this monga title is : 古見同學有交流障礙症漫畫82
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫82/p_1.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫82/p_2.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫82/p_3.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫82/p_4.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫82/p_5.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫82/p_6.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫82/p_7.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫82/p_8.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫82/p_9.jpg
[INFO]Done download monga 古見同學有交流障礙症漫畫82
image page link = http://www.cartoonmad.com/comic/554000832010001.html
this monga total page number = 10
url template : http://web.cartoonmad.com/c8kcs73hv62/5540/083/
http://web.cartoonmad.com/c8kcs73hv62/5540/083/001.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/083/002.jpg
http://web.cartoonmad.com/c8kcs73hv62/5540/083/003.jpg
http://web.cartoonmad.com/c8kcs73hv62/55

this monga title is : 古見同學有交流障礙症漫畫88
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫88/p_1.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫88/p_2.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫88/p_3.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫88/p_4.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫88/p_5.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫88/p_6.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫88/p_7.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫88/p_8.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫88/p_9.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫88/p_10.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫88/p_11.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫88/p_12.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫88/p_13.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫88/p_14.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫88/p_15.jpg
[INFO]Done download page crawl_mongas/古見同學

this monga title is : 古見同學有交流障礙症漫畫93
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫93/p_1.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫93/p_2.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫93/p_3.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫93/p_4.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫93/p_5.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫93/p_6.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫93/p_7.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫93/p_8.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫93/p_9.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫93/p_10.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫93/p_11.jpg
[INFO]Done download monga 古見同學有交流障礙症漫畫93
image page link = http://www.cartoonmad.com/comic/554000942020001.html
this monga total page number = 20
url template : http://web.cartoonmad.com/c8kcs73hv62/5540/094/
http://web.cartoonmad.com/c8kcs73hv62/5540/094/001.jpg
http://web.cartoonmad.com/

[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫97/p_6.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫97/p_7.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫97/p_8.jpg
[INFO]Done download page crawl_mongas/古見同學有交流障礙症漫畫97/p_9.jpg
[INFO]Done download monga 古見同學有交流障礙症漫畫97
Done download all mongas!
